In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from atusfunclib import load_data

In [3]:
# Import all csv data
data_inport = load_data(loc='data')

In [4]:
# Unpack individual dataframes
df, dfactcodes, dfeducodes, dfinccodes, dfagecodes = data_inport[:5]
dfempcodes, dfindcodes, dfraccodes, dfloccodes, dfwhocodes = data_inport[5:]

In [5]:
positiveWL = ['010101', '0102', '0103', '050201',
              '050203', '1101', '1201', '1202',
              '1203', '1204', '1301', '1302', '14']

In [7]:
negoccWL = ['05', '0501', '0504', '1805']

In [8]:
neghomeWL = ['0201', '0202', '0203', '0204', '0209',
             '0301', '0302', '0303', '0304', '0305',
             '0802', '0803']